# Ollama 엔드포인트를 구글 코랩에서 실행

https://stackoverflow.com/questions/77697302/how-to-run-ollama-in-google-colab/77828874#77828874     
https://github.com/ollama/ollama/issues/3215

참고(원본) 유튜브 영상
https://www.youtube.com/watch?v=DVoPHNIKoRA&t=272

In [ ]:
!curl https://ollama.ai/install.sh | sh

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

!pip install pyngrok wget

In [ ]:
# EEVE 모델의 gguf 파일 다운로드
!wget -O ggml-model-Q5_K_M.gguf https://huggingface.co/heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF/resolve/main/ggml-model-Q5_K_M.gguf?download=true

In [ ]:
# Modelfile 다운로드
!wget -O Modelfile https://raw.githubusercontent.com/namkidong98/Ollama-Docker/main/Modelfile

In [12]:
# ngrok의 인증 토큰을 추가
!ngrok config add-authtoken <AuthToken>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )
from IPython.display import clear_output
clear_output()


await asyncio.gather(
run_process(['ollama', 'serve']),
run_process(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header="localhost:11434"']),
run_process(['ollama', 'create', 'EEVE-Korean-10.8B', '-f', 'Modelfile']),
run_process(['ollama', 'list']),
)